In [1]:
import openai

from langsmith import Client
from qdrant_client import QdrantClient

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

### Download an example reference data point from LangSmith


In [2]:
# langsmith client
client = Client()

In [3]:
dataset = client.read_dataset(dataset_name="rag-evaluation-dataset")

In [4]:
dataset

Dataset(name='rag-evaluation-dataset', description='Dataset for evaluating RAG pipeline', data_type=<DataType.kv: 'kv'>, id=UUID('0a6a7d33-d600-4898-bcc3-24f7516f9c46'), created_at=datetime.datetime(2025, 10, 20, 14, 20, 51, 950623, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2025, 10, 20, 14, 20, 51, 950623, tzinfo=datetime.timezone.utc), example_count=38, session_count=0, last_session_start_time=None, inputs_schema=None, outputs_schema=None, transformations=None, metadata={'runtime': {'sdk': 'langsmith-py', 'library': 'langsmith', 'runtime': 'python', 'platform': 'macOS-15.6.1-arm64-arm-64bit', 'sdk_version': '0.4.34', 'runtime_version': '3.12.11', 'langchain_version': None, 'py_implementation': 'CPython', 'langchain_core_version': '0.3.79'}})

In [5]:
list(client.list_examples(dataset_id=dataset.id, limit=30))[28].inputs

{'question': 'I need waterproof Bluetooth headphones for sports. Which options are available?'}

In [6]:
list(client.list_examples(dataset_id=dataset.id, limit=30))[28].outputs

{'ground_truth': 'We have several waterproof Bluetooth headphones suitable for sports: VOESUD Wireless Earbuds (B0B51PZC3L) with IP7 waterproof rating, Bluetooth Headphones (B0C52WNWY6) also with IP7, and the XpacEbank Wireless Bluetooth Headphones (B09Y57K53R) designed for comfort and water resistance.',
 'reference_context_ids': ['B0B51PZC3L', 'B0C52WNWY6', 'B09Y57K53R'],
 'reference_descriptions': ["VOESUD 2022 Upgrade Wireless Earbuds, Bluetooth 5.3 Headphones with Mic,Wireless Headphones Running with IP7 Waterproof Ear Hooks,Touch Control for Sport/WorkAdvanced Bluetooth 5.3 Technology: Q53 Wireless Earbuds use the advanced Bluetooth 5.3 chipset. Use wireless headphones you will getting 3x faster transmission speed and more stable connectivity without signal loss or music dropouts. Whether you are at home, in the office, or on the road, you don't have to worry about disconnection. Thetruly bluetooth headphones make you jump out of limits rather than pulled back by wires. Immersive

In [7]:
reference_input = list(client.list_examples(dataset_id=dataset.id, limit=30))[28].inputs
reference_output = list(client.list_examples(dataset_id=dataset.id, limit=30))[
    28
].outputs

In [8]:
# - use these ref. against what our RAG system is putting out
#


# RAG Pipeline
###### -  let's give ref. input to the RAG Pipeline 


In [9]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [10]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )

    return response.data[0].embedding


def retrieve_data(query, qdrant_client, k=5):
    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "similarity_scores": similarity_scores,
    }


def process_context(context):
    formatted_context = ""

    for id, chunk in zip(
        context["retrieved_context_ids"], context["retrieved_context"]
    ):
        formatted_context += f"- {id}: {chunk}\n"

    return formatted_context


def build_prompt(preprocessed_context, question):
    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt


def generate_answer(prompt):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature=0.5,
    )

    return response.choices[0].message.content


def rag_pipeline(question, qdrant_client, top_k=5):
    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    final_result = {
        "answer": answer,
        "question": question,
        "retrieved_context_ids": retrieved_context["retrieved_context_ids"],
        "retrieved_context": retrieved_context["retrieved_context"],
        "similarity_scores": retrieved_context["similarity_scores"],
    }

    return final_result

In [11]:
rag_pipeline("Can I get some charger?", qdrant_client, top_k=5)

{'answer': 'Yes, there are several chargers available:\n\n1. iPhone Charger Block Cube with 2-pack fast charging data sync lightning cables, compatible with iPhone 14/13/12/11/10/XS/XR/8 Plus/8/7/AirPods and other Apple devices. (Product B0BM961XHH)\n\n2. 45W USB-C Charger compatible with HP, Dell, Lenovo, Acer, Asus, Samsung, Sony, and more, supporting fast charging with multiple voltage outputs. (Product B0B2853GJT)\n\n3. Surge Protector Power Strip with 8 outlets and 4 USB ports (24W PD), which can also serve as a charger for USB devices. (Product B09N7BHBV1)\n\n4. YKZ 3 in 1 Magnetic USB-C to USB-C charging cables (4 pack) supporting PD 60W fast charge for various devices, though you may need a USB-C wall charger to use them. (Product B09NKXF9YQ)\n\nIf you specify the type of charger or device you need it for, I can help you choose the best option.',
 'question': 'Can I get some charger?',
 'retrieved_context_ids': ['B0BM961XHH',
  'B0B2853GJT',
  'B09N7BHBV1',
  'B09XT2BBJF',
  'B

### RAGAS METRICS

In [12]:
from ragas.dataset_schema import SingleTurnSample
from ragas.metrics import (
    IDBasedContextPrecision,
    IDBasedContextRecall,
    Faithfulness,
    ResponseRelevancy,
)

ragas_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
ragas_embeddings = LangchainEmbeddingsWrapper(
    OpenAIEmbeddings(model="text-embedding-3-small")
)

/var/folders/7d/0kh1mw2n62x6t5x8lp0hgjt80000gp/T/ipykernel_977/1862426647.py:4: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use the modern LLM providers instead: from ragas.llms.base import llm_factory; llm = llm_factory('gpt-4o-mini') or from ragas.llms.base import instructor_llm_factory; llm = instructor_llm_factory('openai', client=openai_client)
  ragas_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
/var/folders/7d/0kh1mw2n62x6t5x8lp0hgjt80000gp/T/ipykernel_977/1862426647.py:5: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  ragas_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))


In [15]:
result = rag_pipeline(reference_input["question"], qdrant_client, top_k=5)

In [16]:
result

{'answer': 'The available waterproof Bluetooth headphones suitable for sports are:\n\n1. XpacEbank Wireless Bluetooth Headphone (B09Y57K53R) - Features a lightweight, non-in-ear bone conduction design, waterproof and sweatproof, Bluetooth 5.0, 10 hours playback, and comfortable for long wear during sports or workouts.\n\n2. VOESUD 2022 Upgrade Wireless Earbuds (B0B51PZC3L) - Equipped with Bluetooth 5.3, IPX7 waterproof rating, adjustable silicone earhooks for secure fit during sports, 8 hours playtime per charge with 120 hours total including charging case, touch controls, and noise-cancelling microphones.\n\n3. Wireless Earbud Bluetooth Headphones (B0C52WNWY6) - Bluetooth 5.1 earbuds with IP7 waterproof rating, ergonomic design for comfort during workouts, noise cancelling mic, 6-7 hours playtime, touch controls, and suitable for sports and outdoor activities.\n\nAll three options are waterproof and designed to stay secure and comfortable during sports activities.',
 'question': 'I ne

In [17]:
async def ragas_faithfulness(run, example):
    sample = SingleTurnSample(
        user_input=run["question"],
        response=run["answer"],
        retrieved_contexts=run["retrieved_context"],
    )
    score = Faithfulness(llm=ragas_llm)
    return await score.single_turn_ascore(sample)

In [18]:
await ragas_faithfulness(result, "")

0.8846153846153846

In [19]:
async def ragas_response_relevancy(run, example):
    sample = SingleTurnSample(
        user_input=run["question"],
        response=run["answer"],
        retrieved_contexts=run["retrieved_context"],
    )
    score = ResponseRelevancy(llm=ragas_llm, embeddings=ragas_embeddings)
    return await score.single_turn_ascore(sample)

In [20]:
await ragas_response_relevancy(result, "")

np.float64(0.9061696850294565)

In [21]:
async def ragas_context_precision_id_based(run, example):
    sample = SingleTurnSample(
        retrieved_context_ids=run["retrieved_context_ids"],
        reference_context_ids=example["reference_context_ids"],
    )
    scorer = IDBasedContextPrecision()

    return await scorer.single_turn_ascore(sample)

In [22]:
await ragas_context_precision_id_based(result, reference_output)

0.6

In [23]:
reference_output

{'ground_truth': 'We have several waterproof Bluetooth headphones suitable for sports: VOESUD Wireless Earbuds (B0B51PZC3L) with IP7 waterproof rating, Bluetooth Headphones (B0C52WNWY6) also with IP7, and the XpacEbank Wireless Bluetooth Headphones (B09Y57K53R) designed for comfort and water resistance.',
 'reference_context_ids': ['B0B51PZC3L', 'B0C52WNWY6', 'B09Y57K53R'],
 'reference_descriptions': ["VOESUD 2022 Upgrade Wireless Earbuds, Bluetooth 5.3 Headphones with Mic,Wireless Headphones Running with IP7 Waterproof Ear Hooks,Touch Control for Sport/WorkAdvanced Bluetooth 5.3 Technology: Q53 Wireless Earbuds use the advanced Bluetooth 5.3 chipset. Use wireless headphones you will getting 3x faster transmission speed and more stable connectivity without signal loss or music dropouts. Whether you are at home, in the office, or on the road, you don't have to worry about disconnection. Thetruly bluetooth headphones make you jump out of limits rather than pulled back by wires. Immersive

In [24]:
async def ragas_context_recall_id_based(run, example):
    sample = SingleTurnSample(
        retrieved_context_ids=run["retrieved_context_ids"],
        reference_context_ids=example["reference_context_ids"],
    )
    scorer = IDBasedContextRecall()

    return await scorer.single_turn_ascore(sample)

In [25]:
await ragas_context_recall_id_based(result, reference_output)

1.0